# DARE Deluxe Data Challenge
DARE 2023

In this notebook, we will provide a template for the DARE Deluxe Data Challenge. The main steps covered here will be:

- Load the data
- Provide an overview of what is in the data
- Provide an example of a terribly performing baseline model
- Provide functions to quantify model predictive performance

In [ ]:
# magic
%load_ext autoreload
%autoreload 2
%pdb 1
%matplotlib inline

## Imports and settings
Everything we need to get started.

In [ ]:
import os, sys
import pandas as pd
import matplotlib.pyplot as plt

# data
from functions.data_functions import(load_raw_data) 

## Load the data
Briefly look at the contents and structure of the data.

In [ ]:
process_data = load_raw_data()
process_data.head()

In [ ]:
# lets look at some properties of the data 
print('Number of features: {}'.format(process_data.shape[1]))
print('Number of observations: {}'.format(process_data.shape[0]))

print('Variable names: {}'.format(process_data.columns.tolist()))

process_data.describe()

## Dataset description


In [ ]:
# First of all lets resample hourly taking the mean of each of the process variables
hourly_data = process_data.resample('H').mean()
print('Number of hourly observations: {} of {}'.format(hourly_data.dropna().shape[0],hourly_data.shape[0]))

Training data

In [ ]:
print('Training data is around {:.2f}% of the total data'.format(train_data.shape[0]/hourly_data.shape[0]*100))
print('Training - number of hourly observations: {} of {}'.format(train_data.dropna().shape[0],train_data.shape[0]))

Test data

In [ ]:
# now split the test data - conveniently about 20%
test_data = hourly_data.loc['2017-08-03 21:00':,:]
print('Test data is around {:.2f}% of the total data'.format(test_data.shape[0]/hourly_data.shape[0]*100))
print('Test - number of hourly observations: {} of {}'.format(test_data.dropna().shape[0],test_data.shape[0]))

In [ ]:
# Lets now save the training and test data as hourly mean
train_data.to_csv('data/hourly_train_data.csv')
test_data.to_csv('data/hourly_test_data.csv')